##### imports


In [ ]:
import os

import cv2
import yaml
import numpy as np
from skimage import measure
import pandas as pd

from pathlib import Path
from sklearn.metrics import confusion_matrix as sklearn_confmat
from tqdm import tqdm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

### PATHS


Path to MESSIDOR diagnostics excel: "annotation_all.xls"


In [ ]:
CWD = Path("...")
MESSIDOR_DIAGNOSTIS_XLS = CWD / ".." / "annotation_all.xls"

Path to MAPLES-DR Additionnal data folder


In [ ]:
MAPLES_ADDITIONAL = CWD / "AdditionalData"
MAPLES_DIAGNOSIS_XLS = CWD / "diagnosis.xls"

Ensure output path exists


In [ ]:
os.makedirs("figures/autoseg", exist_ok=True)

### Colors


In [ ]:
WHITE = "#ffffff"

TEAL0 = "#9ec5d4"
TEAL1 = "#75a8ba"
TEAL2 = "#4b8a9f"
TEAL3 = "#277088"
TEAL4 = "#14597e"

PURPLE00 = "#beadd1"
PURPLE0 = "#b8a1d1"
PURPLE1 = "#a17bb8"
PURPLE1bis = "#927dc0"
PURPLE2 = "#8e5ea1"
PURPLE2bis = "#7f5fa9"
PURPLE3 = "#7b3e8a"
PURPLE4 = "#6a1e75"

# Diagnostics


### Load diagnostics


Define columns names and values


In [ ]:
retinologists = ["RetinologistA", "RetinologistB", "RetinologistC"]
gt = "Consensus"
DR_grades = ["R0", "R1", "R2", "R3", "R4A", "R4S"]
ME_grades = ["M0", "M1", "M2"]

Load all diagnostics for ME and DR available in MAPLES-DR.


In [ ]:
DR_diagnostic = pd.read_excel(str(MAPLES_DIAGNOSIS_XLS), sheet_name="DR", index_col="name")
ME_diagnostic = pd.read_excel(str(MAPLES_DIAGNOSIS_XLS), sheet_name="ME", index_col="name")

Load all diagnostics for ME and DR available in MAPLES-DR.


In [ ]:
messidor_diagnostic = pd.read_excel(MESSIDOR_DIAGNOSTIS_XLS)
messidor_diagnostic["name"] = messidor_diagnostic["name"].apply(lambda n: n[:-4])
messidor_diagnostic = messidor_diagnostic.loc[messidor_diagnostic["name"].isin(DR_diagnostic.index)]
messidor_diagnostic = messidor_diagnostic.sort_values(by="name")

messidor_diagnostic = pd.DataFrame(
    {
        "name": messidor_diagnostic["name"],
        "DR": messidor_diagnostic["retinopathy"].map({0: "R0", 1: "R1", 2: "R2", 3: "R3"}),
        "ME": messidor_diagnostic["edema"].map({0: "M0", 1: "M1", 2: "M2"}),
    }
)
messidor_diagnostic.index = messidor_diagnostic["name"]
del messidor_diagnostic["name"]

Define correspondence between MAPLES-DR grades and MESSIDOR grades for DR


In [ ]:
DR_grades_map = {
    "R0": "R0",
    "R1": "R1",
    "R2": "R1",
    "R3": "R2",
    "R4A": "R3",
    "R4S": "R3",
}

Derive quality specific labels


In [ ]:
def grades2quality(grade):
    if isinstance(grade, str):
        return not grade.strip().endswith("6")
    else:
        return grade != -1


DR_quality = DR_diagnostic.copy()
ME_quality = DR_diagnostic.copy()

for r in retinologists + [gt]:
    DR_quality[r] = DR_diagnostic[r].map(grades2quality)
    ME_quality[r] = ME_diagnostic[r].map(grades2quality)

### Grades Distribution


Plots the distribution of grades for ME and DR in MAPLES-DR.

The color of the bars indicate the original grades in the MESSIDOR dataset.


In [ ]:
maples_diagnostics = pd.DataFrame(
    {
        "MAPLES-DR.ME": ME_diagnostic["Consensus"],
        "MAPLES-DR.DR": DR_diagnostic["Consensus"],
        "MESSIDOR.ME": messidor_diagnostic["ME"],
        "MESSIDOR.DR": messidor_diagnostic["DR"],
    }
)
dr_color = {"R0": TEAL0, "R1": TEAL1, "R2": TEAL2, "R3": TEAL3}
me_color = {"M0": PURPLE0, "M1": PURPLE1, "M2": PURPLE2}
histogram_DR = {
    messidor: [
        ((maples_diagnostics["MAPLES-DR.DR"] == maples) & (maples_diagnostics["MESSIDOR.DR"] == messidor)).sum()
        for maples in DR_grades
    ]
    for messidor in dr_color.keys()
}
histogram_ME = {
    messidor: [
        ((maples_diagnostics["MAPLES-DR.ME"] == maples) & (maples_diagnostics["MESSIDOR.ME"] == messidor)).sum()
        for maples in ME_grades
    ]
    for messidor in me_color.keys()
}

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.1, column_widths=[0.64, 0.32])

for dr, color in dr_color.items():
    legend = dict(name=dr, legendgroup="DR", legendgrouptitle_text="<b>MESSIDOR DR</b>")
    styling = dict(marker_color=color, width=0.5)
    if dr == "R3":
        styling |= dict(
            text=[(maples_diagnostics["MAPLES-DR.DR"] == _).sum() for _ in DR_grades],
            textposition="outside",
        )
    fig.add_trace(
        go.Bar(x=DR_grades, y=histogram_DR[dr], **styling, **legend),
        row=1,
        col=1,
    )


for me, color in me_color.items():
    legend = dict(name=me, legendgroup="ME", legendgrouptitle_text="<b>MESSIDOR ME</b>")
    styling = dict(marker_color=color, width=0.5)
    if me == "M2":
        styling |= dict(
            text=[(maples_diagnostics["MAPLES-DR.ME"] == _).sum() for _ in ME_grades],
            textposition="outside",
        )
    fig.add_trace(
        go.Bar(x=ME_grades, y=histogram_ME[me], **styling, **legend),
        row=1,
        col=2,
    )

fig.layout.barmode = "stack"

fig.update_yaxes(
    title="Number of images",
    tickfont={"size": 13},
)
fig.update_yaxes(row=1, col=2, showticklabels=False)
fig.update_xaxes(tickfont={"size": 15})
fig.update_xaxes(row=1, col=1, title={"text": "Diabetic Retinopathy", "font_size": 15})
fig.update_xaxes(row=1, col=2, title={"text": "Macular Eudema", "font_size": 15})

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.0,
        xanchor="center",
        x=0.5,
        bordercolor="#b5aaad",
        borderwidth=0.5,
        grouptitlefont_size=11,
    ),
    height=400,
    width=600,
    template="plotly_white",
    margin={"t": 20, "l": 20, "r": 20, "b": 20},
)

fig.show()

### Retinologists Agreements


In [ ]:
df = pd.DataFrame(
    {
        "ME": ME_diagnostic[gt],
        "DR": DR_diagnostic[gt],
        "ME.nAgree": sum(ME_diagnostic[gt] == ME_diagnostic[r] for r in retinologists),
        "DR.nAgree": sum(DR_diagnostic[gt] == DR_diagnostic[r] for r in retinologists),
    }
)
agreement = {3: "3/3: Consensus", 2: "2/3: Majority", 1: "Deliberation"}
dr_color_nAgree = {3: TEAL4, 2: TEAL2, 1: TEAL0}
me_color_nAgree = {3: PURPLE4, 2: PURPLE2, 1: PURPLE0}
histogram_DR = {
    n: [((df["DR"] == dr) & (df["DR.nAgree"] == n)).sum() for dr in DR_grades] for n in dr_color_nAgree.keys()
}
histogram_ME = {
    n: [((df["ME"] == me) & (df["ME.nAgree"] == n)).sum() for me in ME_grades] for n in me_color_nAgree.keys()
}

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.1, column_widths=[0.64, 0.32])

text_styling = dict(textposition="outside", textfont={"size": 11})

for nAgree, agreeName in agreement.items():
    color = dr_color_nAgree[nAgree]
    legend = dict(name=agreeName, legendgroup="DR", legendgrouptitle_text="<b>DR</b> Agreement")
    styling = dict(marker_color=color, width=0.5, dy=-1)
    if nAgree == 1:
        styling |= dict(
            text=[(maples_diagnostics["MAPLES-DR.DR"] == _).sum() for _ in DR_grades],
            **text_styling,
        )
    fig.add_trace(
        go.Bar(x=DR_grades, y=histogram_DR[nAgree], **styling, **legend),
        row=1,
        col=1,
    )

for nAgree, agreeName in agreement.items():
    color = me_color_nAgree[nAgree]
    legend = dict(name=agreeName, legendgroup="ME", legendgrouptitle_text="<b>ME</b> Agreement")
    styling = dict(marker_color=color, width=0.5)
    if nAgree == 1:
        styling |= dict(
            text=[(maples_diagnostics["MAPLES-DR.ME"] == _).sum() for _ in ME_grades],
            **text_styling,
        )
    fig.add_trace(
        go.Bar(x=ME_grades, y=histogram_ME[nAgree], **styling, **legend),
        row=1,
        col=2,
    )

fig.layout.barmode = "stack"

fig.update_yaxes(range=[0, 175])
fig.update_yaxes(
    title={"text": "Number of images", "font_size": 15},
    tickfont={"size": 13},
    row=1,
    col=1,
)
fig.update_yaxes(row=1, col=2, showticklabels=False)
fig.update_xaxes(tickfont={"size": 13})
fig.update_xaxes(row=1, col=1, title={"text": "<b>Diabetic Retinopathy</b>", "font_size": 15})
fig.update_xaxes(row=1, col=2, title={"text": "<b>Macular Eudema</b>", "font_size": 15})

fig.update_layout(
    bargap=0,
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.0,
        xanchor="center",
        x=0.5,
        bordercolor="#b5aaad",
        borderwidth=0.5,
        grouptitlefont_size=11,
    ),
    height=300,
    width=600,
    template="plotly_white",
    margin={"t": 20, "l": 5, "r": 10, "b": 20},
)

fig.show()
fig.write_image("figures/DR_post_dist.pdf")

DR_resume = pd.DataFrame(
    {
        dr: {agreeName: histogram_DR[nAgree][i_dr] for nAgree, agreeName in agreement.items()}
        for i_dr, dr in enumerate(DR_grades)
    }
)
DR_resume["Total"] = DR_resume.sum(axis=1)
display(DR_resume)

ME_resume = pd.DataFrame(
    {
        me: {agreeName: histogram_ME[nAgree][i_me] for nAgree, agreeName in agreement.items()}
        for i_me, me in enumerate(ME_grades)
    }
)
ME_resume["Total"] = ME_resume.sum(axis=1)

display(ME_resume)

# Lesions Count


### Prepare data


In [ ]:
with open(MAPLES_ADDITIONAL / "biomarkers-export-config.yaml", "r") as f:
    config = yaml.load(f, yaml.Loader)

ANNOTATIONS_PATH = MAPLES_ADDITIONAL / "annotations"
PREANNOTATIONS_PATH = MAPLES_ADDITIONAL / "preannotations"

bright = config["multiclass"]["BrightLesions"]
bright_preannotations = ["Exudates"]
red = config["multiclass"]["RedLesions"]
red_preannotations = ["Microaneurysms", "Hemorrhages"]
anatomical = config["multiclass"]["OtherAnatomical"]
vessel = "Vessels"

imgnames = [f.name for f in (ANNOTATIONS_PATH / "OpticDisc").iterdir() if f.suffix == ".png"]

In [ ]:
lesions = [
    "Microaneurysms",
    "Hemorrhages",
    "Neovascularization",
    "Exudates",
    "CottonWoolSpots",
    "Drusens",
]


def lesions_info(img: str) -> dict:  # Path: plots.ipynb
    img = cv2.imread(img).mean(axis=2) > 127
    # Count lesions
    labels = measure.label(img)
    lesions_count = labels.max()
    # Compute average lesions area
    lesions_sizes = [np.sum(labels == i) for i in range(1, lesions_count + 1)]
    return {
        "count": lesions_count,
        "area": img.sum(),
        "med_size": np.median(lesions_sizes),
    }


lesion_data = []
for lesion in lesions:
    lesion_path = ANNOTATIONS_PATH / lesion
    for img_path in tqdm(lesion_path.iterdir()):
        if img_path.suffix == ".png":
            lesion_data += [lesions_info(str(img_path)) | {"name": img_path.stem, "lesion": lesion}]
all_dataset_long = pd.DataFrame(lesion_data).merge(maples_diagnostics, left_on="name", right_index=True)

In [ ]:
def fuse_labels(img_name, folders_path):
    labels = [cv2.imread(str(folder / img_name), cv2.IMREAD_GRAYSCALE) for folder in folders_path]
    fused = np.zeros_like(labels[0], dtype=np.uint8)
    for i, label in enumerate(labels):
        fused[label > 127] = i + 1
    return fused


def single_image_correction_stat(img_name, annotation_labels, preannotation_labels=()):
    annotations = fuse_labels(img_name, [ANNOTATIONS_PATH / l for l in annotation_labels])
    if len(preannotation_labels):
        preannotations = fuse_labels(img_name, [PREANNOTATIONS_PATH / l for l in preannotation_labels])
        return sklearn_confmat(
            annotations.flatten(),
            preannotations.flatten(),
            labels=np.arange(len(annotation_labels) + 1),
        )
    else:
        return np.array([[(_ == annotations).sum()] for _ in range(len(annotation_labels) + 1)])


def correction_stat(annotation_labels, preannotation_labels=(), desc=None):
    cm = None
    for img in tqdm(imgnames, desc=desc):
        cm_img = single_image_correction_stat(img, annotation_labels, preannotation_labels)
        if cm is None:
            cm = cm_img
        else:
            cm += cm_img
    df = pd.DataFrame(
        cm[:, : len(preannotation_labels) + 1],
        columns=("back",) + tuple(preannotation_labels),
        index=("back",) + tuple(annotation_labels),
    )
    return df

In [ ]:
bright_cm = correction_stat(bright, bright_preannotations)
bright_cm

In [ ]:
red_cm = correction_stat(red, red_preannotations)
red_cm

In [ ]:
vessels_cm = correction_stat([vessel], [vessel], desc="Vessels")
vessels_cm

In [ ]:
anatomical_stat = correction_stat(anatomical, desc="Anatomical")
anatomical_stat

### Plots


In [ ]:
ADDED = "Manual Annotations"
ERASED = "Erased Preannotations"


def PRE(lesion):
    return lesion.strip().capitalize() + " Preannotations"


TOTAL = "Total"

names_map = {
    "Microaneurysms": "Microaneurysms",
    "Hemorrhages": "Hemorrhages",
    "Neovascularization": "Neovascularization",
    "RedUncertains": "Uncertain Red Lesions",
    "Exudates": "Exudates",
    "CottonWoolSpots": "Cotton Wool Spots",
    "Drusens": "Drusens",
    "BrightUncertains": "     Uncertain Bright Lesions",
}
red = [lesion for lesion in names_map.keys() if lesion in red]
bright = [lesion for lesion in names_map.keys() if lesion in bright]
red_labels = list(map(names_map.get, red))
bright_labels = list(map(names_map.get, bright))

In [ ]:
total_px = 1500 * 1500 * 200
# coef = 1/total


def select_erased(for_lesions, conf_mat):
    selected_data = np.array([conf_mat[l]["back"] if l in conf_mat else 0 for l in for_lesions])
    return selected_data


def select_preannotated(preannotation, for_lesions, conf_mat):
    selected_data = np.array([conf_mat[preannotation][l] if l in conf_mat[preannotation] else 0 for l in for_lesions])
    return selected_data


def select_added(for_lesions, conf_mat):
    selected_data = np.array([conf_mat["back"][l] if l in conf_mat["back"] else 0 for l in for_lesions])
    return selected_data


def compute_total(for_lesions, conf_mat):
    selected_data = np.array([conf_mat.T[l].sum() if l in conf_mat.T else 0 for l in for_lesions])
    return selected_data


# === Red lesions ===
red_data = {
    ERASED: select_erased(red, red_cm),
    PRE("microaneurysms"): select_preannotated("Microaneurysms", red, red_cm),
    PRE("hemorrhages"): select_preannotated("Hemorrhages", red, red_cm),
    ADDED: select_added(red, red_cm),
    TOTAL: compute_total(red, red_cm),
}

# === Bright lesions ===
bright_data = {
    ERASED: select_erased(bright, bright_cm),
    PRE("exudates"): select_preannotated("Exudates", bright, bright_cm),
    ADDED: select_added(bright, bright_cm),
    TOTAL: compute_total(bright, bright_cm),
}

# === Vessels ===
vessels_data = {
    ERASED: select_erased(["Vessels"], vessels_cm)[0],
    PRE("vessels"): select_preannotated("Vessels", ["Vessels"], vessels_cm)[0],
    ADDED: select_added(["Vessels"], vessels_cm)[0],
    TOTAL: compute_total(["Vessels"], vessels_cm)[0],
}

In [ ]:
fig = make_subplots(
    cols=1,
    rows=2,
    vertical_spacing=0.03,
    shared_xaxes=True,
)


def format_percent(x):
    return f"{abs(x)*1000:.2f}‰"


def format_px(x):
    x = abs(x) / 200
    if x > 1e3:
        return f"<b>{x/1000:.1f}</b> kpx"
    else:
        return f"<b>{x:.0f}</b> px"


bar_style = dict(
    width=0.75,
    orientation="h",
)

# === Red lesions ===
red_color = {
    ADDED: PURPLE4,
    PRE("microaneurysms"): PURPLE2,
    PRE("hemorrhages"): PURPLE2bis,
    ERASED: PURPLE00,
}

red_style = (
    dict(
        legendgroup="red",
        legendgrouptitle_text="Red Lesions",
    )
    | bar_style
)
offset = np.zeros_like(red_data["Total"] / total_px)
for src in reversed(red_color.keys()):
    color = red_color[src]
    legendrank = list(red_color.keys()).index(src)
    if src == ERASED:
        bar = go.Bar(y=red_labels, x=-red_data[src] / total_px)
        bar.text = list(map(format_px, red_data[ERASED]))
        bar.update(textfont={"color": WHITE, "size": 10}, insidetextanchor="middle")
    else:
        x = red_data[src] / total_px
        bar = go.Bar(y=red_labels, x=x, base=offset)
        offset += x
        if src == ADDED:
            bar.text = list(map(format_px, red_data[ADDED]))
            bar.update(textfont={"color": WHITE, "size": 10}, insidetextanchor="middle")
    bar.update(name=src, marker_color=color, legendrank=legendrank, **red_style)
    fig.add_trace(bar, row=1, col=1)

for i, l in enumerate(red_labels):
    x = red_data["Total"][i] / total_px
    fig.add_annotation(
        text=format_percent(x),
        x=x,
        y=l,
        xref="x",
        xanchor="left",
        yref="y",
        showarrow=False,
        font={"size": 11, "color": PURPLE4},
        row=1,
        col=1,
    )

# === Bright lesions ===
bright_color = {
    ADDED: TEAL4,
    PRE("exudates"): TEAL2,
    ERASED: TEAL0,
}

bright_style = (
    dict(
        legendgroup="bright",
        legendgrouptitle_text="Bright Lesions",
    )
    | bar_style
)

offset = np.zeros_like(bright_data["Total"] / total_px)
for src in reversed(bright_color.keys()):
    color = bright_color[src]
    legendrank = list(bright_color.keys()).index(src)
    if src == ERASED:
        bar = go.Bar(y=bright_labels, x=-bright_data[src] / total_px)
        bar.text = list(map(format_px, bright_data[ERASED]))
        bar.update(textfont={"color": WHITE, "size": 10}, insidetextanchor="middle")
    else:
        x = bright_data[src] / total_px
        bar = go.Bar(y=bright_labels, x=x, base=offset)
        offset += x
        if src == ADDED:
            bar.text = list(map(format_px, bright_data[ADDED]))
            bar.update(textfont={"color": WHITE, "size": 10}, insidetextanchor="middle")
    bar.update(marker_color=color, name=src, legendrank=legendrank, **bright_style)
    fig.add_trace(bar, row=2, col=1)

for i, l in enumerate(bright_labels):
    x = bright_data["Total"][i] / total_px
    fig.add_annotation(
        text=format_percent(x),
        x=x,
        xref="x",
        xanchor="left",
        y=l,
        yref="y",
        showarrow=False,
        font={"size": 11, "color": TEAL4},
        row=2,
        col=1,
    )


# === Axis Styling ===
xaxes_style = dict(
    tickfont={"size": 11},
    tickvals=[-1.5e-3, -1e-3, -0.5e-3, 0, 0.5e-3, 1e-3, 1.5e-3, 2e-3, 2.5e-3, 3e-3],
    ticktext=["", "", "", "0", "", "1‰", "", "2‰", "", "3‰"],
    range=[-1.7e-3, 2.8e-3],
)
fig.update_xaxes(**xaxes_style, showticklabels=False, row=1, col=1)
fig.update_xaxes(**xaxes_style, row=2, col=1)

xtitle_standoff = -0.145
fig.add_annotation(
    text="Final Annotated Area per Image",
    x=0.15e-3,
    y=xtitle_standoff,
    xref="x",
    xanchor="left",
    yref="paper",
    showarrow=False,
    font=dict(size=15),
)
fig.add_annotation(
    text=ERASED,
    x=-0.15e-3,
    y=xtitle_standoff,
    xref="x",
    xanchor="right",
    yref="paper",
    showarrow=False,
    font=dict(size=13, color="#888"),
)

fig.update_yaxes(tickfont={"size": 11}, autorange="reversed")
fig.add_annotation(
    text="<b>Lesions</b>",
    x=-0.25,
    y=0.5,
    xref="paper",
    yref="paper",
    showarrow=False,
    font=dict(size=15),
    textangle=-90,
)

# === General Styling ===
fig.layout.barmode = "overlay"
fig.layout.legend.update(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="left",
    x=0.1,
    bordercolor="#b5aaad",
    borderwidth=0.5,
    font_size=11,
)
fig.update_layout(
    height=400,
    width=750,
    template="plotly_white",
    margin={"t": 0, "l": 20, "r": 30, "b": 35},
)

fig.show()
fig.write_image("figures/diff_lesion.pdf")

In [ ]:
fig = make_subplots(cols=2, rows=1, horizontal_spacing=0.2, column_widths=[0.25, 0.75])

bar_style = dict(
    width=0.7,
)


def format_px(x):
    x = abs(x) / 200
    if x > 1e3:
        return f"<b>{x/1000:.0f}</b> kpx"
    else:
        return f"{x:.0f} px"


# === Vessels ===
vessels_color = {
    ADDED: PURPLE4,
    PRE("vessels"): PURPLE2,
    # ERASED: PURPLE00,
}
vessels_style = (
    dict(
        legendgroup="vessels",
        legendgrouptitle_text="Vessels",
    )
    | bar_style
)

offset = 0.0
for src in reversed(vessels_color.keys()):
    color = vessels_color[src]
    legendrank = list(vessels_color.keys()).index(src)
    if src == ERASED:
        bar = go.Bar(x=["Vessels"], y=[-vessels_data[src] / total_px])
    else:
        y = vessels_data[src] / total_px
        bar = go.Bar(x=["Vessels"], y=[y], base=[offset])
        offset += y
        if src == ADDED:
            bar.text = [format_px(vessels_data[ADDED])]
            bar.update(textfont={"color": WHITE, "size": 10}, insidetextanchor="middle")
    bar.update(marker_color=color, name=src, legendrank=legendrank, **vessels_style)
    fig.add_trace(bar, col=1, row=1)

total_vessels = vessels_data["Total"] / total_px
fig.add_annotation(
    text=f"{total_vessels*100:.1f}%",
    x="Vessels",
    xref="x",
    xanchor="center",
    y=total_vessels,
    yref="y",
    yanchor="bottom",
    showarrow=False,
    font={"size": 11, "color": PURPLE4},
    col=1,
    row=1,
)


# === Others Anatomical ===
anatomical_names = {
    "OpticDisc": "Optic Disc",
    "OpticCup": "Optic Cup",
    "Macula": "Macula",
}
anatomical_color = {
    "OpticDisc": TEAL4,
    "OpticCup": TEAL4,
    "Macula": TEAL4,
}

anatomical_style = dict(showlegend=False) | bar_style

for i, src in enumerate(anatomical_names.keys()):
    color = anatomical_color[src]
    y = anatomical_stat["back"][src] / total_px
    bar = go.Bar(x=[anatomical_names[src]], y=[y])
    bar.update(marker_color=color, name=ADDED, **anatomical_style)

    bar.text = [format_px(anatomical_stat["back"][src])]
    bar.update(textfont={"color": WHITE, "size": 10}, insidetextanchor="middle")

    if i == 0:
        bar.update(
            legendgroup="anatomical",
            legendgrouptitle_text="Other Structures",
            showlegend=True,
        )

    fig.add_trace(bar, col=2, row=1)

    fig.add_annotation(
        text=f"{y*1000:.1f}‰",
        x=anatomical_names[src],
        xref="x",
        xanchor="center",
        y=y,
        yref="y",
        yanchor="bottom",
        showarrow=False,
        font={"size": 11, "color": color},
        col=2,
        row=1,
    )


# === Axis Styling ===
fig.update_xaxes(
    tickfont={"size": 12},
    tickangle=-35,
)

fig.update_yaxes(
    tickfont={"size": 11},
    range=[0, 14e-2],
    tickvals=np.linspace(0, 16e-2, 5, endpoint=True),
    ticktext=["0%", "4%", "8%", "12%", "16%"],
    tickformat=".0%",
    title=dict(text="Final Annotated Area", standoff=10, font_size=15),
    col=1,
    row=1,
)

fig.update_yaxes(
    tickfont={"size": 11},
    range=[0, 17.5e-3],
    tickvals=[0, 5e-3, 10e-3, 15e-3, 20e-3],
    ticktext=["0‰", "5‰", "10‰", "15‰", "20‰"],
    side="right",
    # title=dict(text="Final Annotated Area", standoff=0, font_size=15),
    col=2,
    row=1,
)

fig.add_annotation(
    text="<b>Anatomical Structure</b>",
    x=0,
    xref="paper",
    y=-0.32,
    yref="paper",
    showarrow=False,
    font=dict(size=15),
)


# === General Styling ===
fig.layout.barmode = "overlay"
fig.layout.legend.update(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="left",
    x=0.05,
    bordercolor="#b5aaad",
    borderwidth=0.5,
    font_size=11,
    tracegroupgap=0,
)

fig.update_layout(
    height=400,
    width=250,
    template="plotly_white",
    margin={"t": 0, "l": 20, "r": 20, "b": 65},
)


fig.show()
fig.write_image("figures/diff_anatomical.pdf")

In [ ]:
group_by = "MAPLES-DR.DR"

all_dataset_long = all_dataset_long.sort_values(by=group_by)
all_dataset_long = all_dataset_long.sort_values(by="lesion", key=lambda x: x.map(lesions.index))
all_dataset_long["area (%)"] = all_dataset_long["area"] / (1500 * 1500)

dr_color = {
    "R0": TEAL0,
    "R1": TEAL1,
    "R2": TEAL2,
    "R3": TEAL3,
    "R4A": PURPLE2,
}


generic_opt = dict(
    boxpoints=False,
)

fig1 = go.Figure()
for i, (dr, color) in enumerate(dr_color.items()):
    data = all_dataset_long.loc[all_dataset_long[group_by] == dr]
    opt = dict(marker_color=color) | generic_opt
    opt |= dict(name=dr)
    fig1.add_trace(go.Box(x=data["lesion"], y=data["count"], text=data["count"].mean(), **opt))
fig1.update_yaxes(
    title="Lesions count per image",
)


fig2 = go.Figure()
for i, (dr, color) in enumerate(dr_color.items()):
    data = all_dataset_long.loc[all_dataset_long[group_by] == dr]
    opt = dict(marker_color=color) | generic_opt
    opt |= dict(name=dr, showlegend=False)
    fig2.add_trace(
        go.Box(x=data["lesion"], y=data["area (%)"], **opt),
    )

fig2.update_yaxes(
    title="Total lesion area per image",
    tickformat=".0%",
)


def fig_format_and_show(fig):
    fig.layout.boxmode = "group"
    fig.update_layout(
        legend=dict(
            title=dict(text="<b>Diabetic Retinopathy Grade</b>"),
            orientation="h",
            yanchor="top",
            y=1.1,
            xanchor="center",
            x=0.5,
            bordercolor="#b5aaad",
            borderwidth=0.5,
            grouptitlefont_size=11,
        ),
        title=dict(x=0.5, y=1, xanchor="center", yanchor="top"),
        height=300,
        width=1000,
        template="plotly_white",
        margin={"t": 20, "l": 20, "r": 20, "b": 20},
    )
    fig.update_yaxes(tickfont_size=13, title_font_size=15)
    fig.update_xaxes(tickfont_size=15)


fig_format_and_show(fig1)
fig_format_and_show(fig2)
fig2.update_layout(height=260)

fig1.show()
fig2.show()
fig1.write_image("figures/lesions_dist1.pdf")
fig2.write_image("figures/lesions_dist2.pdf")

## Automatic Sementic Segmentation


In [ ]:
perfs = pd.DataFrame(
    [
        {
            "type": "Microaneurysms",
            "Precision": 0.59,
            "Recall": 0.52,
            "IoU": 0.38,
            "Avg. Pr.": 0.56,
        },
        {
            "type": "Hemorrhages",
            "Precision": 0.58,
            "Recall": 0.44,
            "IoU": 0.34,
            "Avg. Pr.": 0.51,
        },
        {
            "type": "Exudates",
            "Precision": 0.75,
            "Recall": 0.78,
            "IoU": 0.62,
            "Avg. Pr.": 0.78,
        },
        {
            "type": "Vessels",
            "Precision": 0.87,
            "Recall": 0.79,
            "IoU": 0.71,
            "Avg. Pr.": 0.86,
        },
        {
            "type": "Macula",
            "Precision": 0.67,
            "Recall": 0.55,
            "IoU": 0.43,
            "Avg. Pr.": 0.61,
        },
        {
            "type": "Optic Disc",
            "Precision": 0.70,
            "Recall": 0.81,
            "IoU": 0.60,
            "Avg. Pr.": 0.77,
        },
        {
            "type": "Optic Cup",
            "Precision": 0.42,
            "Recall": 0.92,
            "IoU": 0.41,
            "Avg. Pr.": 0.68,
        },
    ]
)

In [ ]:
colors = {
    "Microaneurysms": PURPLE4,
    "Hemorrhages": PURPLE4,
    "Exudates": PURPLE2,
    "Vessels": TEAL4,
    "Macula": TEAL1,
    "Optic Disc": TEAL2,
    "Optic Cup": TEAL2,
}
left_biomarkers = ["Microaneurysms", "Exudates", "Optic Disc", "Macula"]

metrics = {"Avg. Pr.": 0, "Precision": 90, "IoU": 180, "Recall": 270}
metrics = {"Precision": 0, "Avg. Pr.": 60, "IoU": 120, "Recall": 180}


def smooth_spline(thetas, radi, close=False, n=100):
    t0 = thetas[0]
    theta = []

    if close:
        thetas = thetas + [360]
        radi = list(radi) + [radi[0]]

    interpolation = scinterp.CubicSpline(thetas, radi, bc_type="periodic" if close else "clamped")

    for t in thetas[1:]:
        theta += [np.linspace(t0, t, n)]
        t0 = t
    theta = np.concatenate(theta)

    return theta, interpolation(theta)


metrics_thetas = list(metrics.values())
metrics_name = list(metrics.keys())
metrics_bold = list(f"<b>{m}</b>" for m in metrics)
metrics_bold[0] = " " * 5 + metrics_bold[0] + " " * 5
metrics_bold[3] = " " * 5 + metrics_bold[3] + " " * 5


for i, (type, color) in enumerate(colors.items()):
    fig = go.Figure()
    data = perfs.loc[perfs["type"] == type]
    RIGHT = type not in left_biomarkers

    if RIGHT:
        thetas = metrics_thetas
    else:
        thetas = [_ + 90 for _ in metrics_thetas]
    metrics_value = data[metrics_name].values[0]
    smooth_theta, smooth_r = smooth_spline(metrics_thetas, metrics_value, close=False)
    fig.add_trace(
        go.Scatterpolar(
            theta=smooth_theta,
            r=smooth_r,
            mode="lines",
            fill="toself",
            line=dict(width=0.5, color=color),
        )
    )
    fig.add_trace(
        go.Scatterpolar(
            theta=metrics_thetas,
            r=metrics_value,
            mode="markers",
            marker=dict(
                color=color,
                size=4,
            ),
        )
    )
    fig.add_trace(
        go.Scatterpolar(
            theta=metrics_thetas,
            r=[r + offset for r, offset in zip(metrics_value, [0.1, 0.05, 0.05, 0.1])],
            text=[f"{_:.0%}" for _ in metrics_value],
            textposition="middle right" if RIGHT else "middle left",
            mode="text",
            textfont=dict(
                color=color,
            ),
        )
    )

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                showline=False,
                range=[0, 1.15],
                angle=-90 if RIGHT else 90,
                showticklabels=False,
                # tickangle=-90 if RIGHT else 90,
                # tickformat=" .1",
                # tickfont=dict(size=11, color="#555"),
                tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1],
            ),
            angularaxis=dict(
                showline=False,
                tickfont_size=13,
                rotation=90,
                direction="clockwise" if RIGHT else "counterclockwise",
                tickvals=metrics_thetas,
                ticktext=metrics_bold,
                gridcolor="#ddd",
                gridwidth=2,
            ),
            sector=[-90, +90] if RIGHT else [90, 270],
        ),
        showlegend=False,
        height=280,
        width=200,
        template="plotly_white",
        margin={"t": 20, "l": 32 if RIGHT else 50, "r": 50 if RIGHT else 32, "b": 20},
    )

    if i == 0:
        fig.update_layout(
            polar_radialaxis=dict(
                showticklabels=True,
                tickangle=-90 if RIGHT else 90,
                tickformat=" .0%",
                tickfont=dict(size=11, color="#555"),
            ),
        )
    fig.show()
    fig.write_image(f"figures/autoseg/{type.replace(' ', '')}.pdf")

In [ ]:
group_by = "MAPLES-DR.DR"

all_dataset_long = all_dataset_long.sort_values(by=group_by)
all_dataset_long = all_dataset_long.sort_values(by="lesion", key=lambda x: x.map(lesions.index))
all_dataset_long["area (%)"] = all_dataset_long["area"] / (1500 * 1500)

dr_color = {
    "R0": TEAL0,
    "R1": TEAL1,
    "R2": TEAL2,
    "R3": TEAL3,
    "R4A": PURPLE2,
}


generic_opt = dict(
    boxpoints=False,
)

fig1 = go.Figure()
for i, (dr, color) in enumerate(dr_color.items()):
    data = all_dataset_long.loc[all_dataset_long[group_by] == dr]
    opt = dict(marker_color=color) | generic_opt
    opt |= dict(name=dr)
    fig1.add_trace(go.Box(x=data["lesion"], y=data["count"], text=data["count"].mean(), **opt))
fig1.update_yaxes(
    title="Lesions count per image",
)


fig2 = go.Figure()
for i, (dr, color) in enumerate(dr_color.items()):
    data = all_dataset_long.loc[all_dataset_long[group_by] == dr]
    opt = dict(marker_color=color) | generic_opt
    opt |= dict(name=dr, showlegend=False)
    fig2.add_trace(
        go.Box(x=data["lesion"], y=data["area (%)"], **opt),
    )

fig2.update_yaxes(
    title="Total lesion area per image",
    tickformat=".0%",
)


def fig_format_and_show(fig):
    fig.layout.boxmode = "group"
    fig.update_layout(
        legend=dict(
            title=dict(text="<b>Diabetic Retinopathy Grade</b>"),
            orientation="h",
            yanchor="top",
            y=1.1,
            xanchor="center",
            x=0.5,
            bordercolor="#b5aaad",
            borderwidth=0.5,
            grouptitlefont_size=11,
        ),
        title=dict(x=0.5, y=1, xanchor="center", yanchor="top"),
        height=350,
        width=1000,
        template="plotly_white",
        margin={"t": 20, "l": 20, "r": 20, "b": 20},
    )
    fig.update_yaxes(tickfont_size=13, title_font_size=15)
    fig.update_xaxes(tickfont_size=15)


fig_format_and_show(fig1)
fig_format_and_show(fig2)
fig2.update_layout(height=320)

fig1.show()
fig2.show()
fig1.write_image("figures/lesions_dist1.pdf")
fig2.write_image("figures/lesions_dist2.pdf")